<a href="https://colab.research.google.com/github/lnpetrova/comp_ling/blob/master/CL_hw8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install pymorphy2[fast]

In [3]:
import json, os
import pandas as pd
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
import numpy as np
from pymorphy2 import MorphAnalyzer
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
morph = MorphAnalyzer()
stops = set(stopwords.words('russian'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
from google.colab import files
uploaded = files.upload()

Saving hw8_data.zip to hw8_data.zip


In [4]:
!unzip hw8_data.zip

Archive:  hw8_data.zip
   creating: hw8_data/
  inflating: hw8_data/russia_today_7.jsonlines  
  inflating: hw8_data/russia_today_6.jsonlines  
  inflating: hw8_data/russia_today_5.jsonlines  
  inflating: hw8_data/russia_today_4.jsonlines  
  inflating: hw8_data/russia_today_3.jsonlines  
  inflating: hw8_data/russia_today_2.jsonlines  
  inflating: hw8_data/russia_today_1.jsonlines  
  inflating: hw8_data/russia_today_0.jsonlines  
  inflating: hw8_data/ng_1.jsonlines  
  inflating: hw8_data/ng_0.jsonlines  
  inflating: hw8_data/habrahabr_3.jsonlines  
  inflating: hw8_data/habrahabr_2.jsonlines  
  inflating: hw8_data/habrahabr_1.jsonlines  
  inflating: hw8_data/habrahabr_0.jsonlines  
  inflating: hw8_data/cyberleninka_4.jsonlines  
  inflating: hw8_data/cyberleninka_3.jsonlines  
  inflating: hw8_data/cyberleninka_2.jsonlines  
  inflating: hw8_data/cyberleninka_1.jsonlines  
  inflating: hw8_data/cyberleninka_0.jsonlines  


In [4]:
files = [os.path.join('hw8_data/.', file) for file in os.listdir('hw8_data/.')]

In [5]:
pd.set_option('display.max_colwidth', 1000)

In [6]:
data = pd.concat([pd.read_json(file, lines=True) for file in files][:5], axis=0, ignore_index=True)

In [7]:
data.shape

(4988, 6)

In [10]:
data.head(5)

,keywords,title,summary,url,content,abstract
0,"[ангела меркель, владимир путин, днр, джон маккейн, дмитрий песков, дональд трамп, донбасс, донецк, киев, лнр, луганск, обсе, переговоры в минске, пётр порошенко, сша, украина, украина]",Сила от бессилия: что стоит за обострением конфликта в Донбассе,"Президент Украины Пётр Порошенко прервал визит в Германию из-за обстрелов в Авдеевке. Об этом сообщил его пресс-секретарь Святослав Цеголко. Ранее украинский лидер отправился с визитом в Германию на фоне беспрецедентного обострения конфликта в Донбассе, где только за последние сутки ВСУ около двух тысяч раз нарушили режим прекращения огня. По какой причине происходит эскалация конфликта, почему молчит миссия ОБСЕ и откуда ждать помощи Донбассу — в материале RT.",https://russian.rt.com/ussr/article/354753-donbass-obostrenie-prichiny,"Канцлер Германии Ангела Меркель в ходе брифинга в преддверии переговоров с украинским президентом заявила, что на востоке Украины не действует режим прекращения огня.\n«Опять узнали, что режима прекращения огня фактически не существует, что снова гибнут солдаты. Ситуация на линии разграничения вызывает беспокойство», — отметила Меркель.\nСам Порошенко заявил, что Украина остаётся преданной минским договорённостям, и призвал страны Запада к продлению и усилению антироссийских санкций, если не будет достигнут прогресс в реализации Минских соглашений по Донбассу.\n Reuters \nЭти заявления звучат по меньшей мере странно на фоне того, что последние несколько дней происходит беспрецедентное обострение обстановки в Донбассе, о чём сам же Порошенко не преминул сообщить председателю бундестага Норберту Ламмерту.\n«Они пришли учиться, а не страдать от обстрелов»\nЗаместитель начальника управления народной милиции ДНР, полковник Эдуард Басурин во время брифингов сообщил, что за последние трое...",NaN
1,"[авиаудары, авиация, армия, белоруссия, боевики, вкс россии, военная техника, военнослужащие, диверсия, оборона, россия, союзники, терроризм, учения, армия]",Завершающий удар: войска Союзного государства начали второй этап учений «Запад-2017»,"Минобороны РФ объявило о начале второго этапа российско-белорусских стратегических учений «Запад-2017». На первом этапе, который прошёл с 14 по 16 сентября, военнослужащие Союзного государства отразили нападение условного противника и взяли под защиту ключевые объекты инфраструктуры. Второй этап предполагает окружение диверсионных групп и их последующее уничтожение. О том, чем успели запомниться стратегические учения, — в материале RT.",https://russian.rt.com/russia/article/431010-ucheniya-zapad-rossiya-belorussiya,"Российские и белорусские войска успешно завершили первый этап стратегических учений «Запад-2017». 17 сентября военнослужащие двух стран приступили к выполнению задач в рамках второго этапа. Условные боевые действия развернутся на полигоне Лужский (Ленинградская область), на полигоне Борисовский (Белоруссия), а также в акватории Балтийского моря.\n Также по теме \n Запад против «Запада»: почему страны НАТО опасаются российско-белорусских учений \n Российские и белорусские военнослужащие начали стратегические учения «Запад-2017». Согласно сценарию войска Союзного государства... \nКак уточняет Минобороны, «силами региональной группировки будут проведены розыгрыши практических действий по ведению манёвренной обороны, переходу к наступательным действиям и завершению разгрома противника». Основная часть учений пройдёт 18 и 20 сентября.\n16 сентября белорусское командование и органы военного управления региональной группировки войск приступили к планированию совместной операции по блокиро...",NaN
2,"[англия, артём дзюба, бразилия, бундеслига, в россии, георгий джикия, германия, испания, италия, нигерия, пфк цска, примера, рфпл, россия, россияне, севилья, сербия, серия а, спорт, спортсмен, трансферы, фк динамо, фк зенит, фк краснодар, фк рубин, фк спартак, фк тосно, фк уфа, фёдор смолов, хорватия, чемпионат россии, футбол]","Дзюба в «Зените-2», первая покупка ЦСКА за три г

In [8]:
def evaluate(true_kws, predicted_kws):
    assert len(true_kws) == len(predicted_kws)
    
    precisions = []
    recalls = []
    f1s = []
    jaccards = []
    
    for i in range(len(true_kws)):
        true_kw = set(true_kws[i])
        predicted_kw = set(predicted_kws[i])
        
        tp = len(true_kw & predicted_kw)
        union = len(true_kw | predicted_kw)
        fp = len(predicted_kw - true_kw)
        fn = len(true_kw - predicted_kw)
        
        if (tp+fp) == 0:
            prec = 0
        else:
            prec = tp / (tp + fp)
        
        if (tp+fn) == 0:
            rec = 0
        else:
            rec = tp / (tp + fn)
        if (prec+rec) == 0:
            f1 = 0
        else:
            f1 = (2*(prec*rec))/(prec+rec)
            
        jac = tp / union
        
        precisions.append(prec)
        recalls.append(rec)
        f1s.append(f1)
        jaccards.append(jac)
    print('Precision - ', round(np.mean(precisions), 2))
    print('Recall - ', round(np.mean(recalls), 2))
    print('F1 - ', round(np.mean(f1s), 2))
    print('Jaccard - ', round(np.mean(jaccards), 2))

In [9]:
from string import punctuation
from nltk.corpus import stopwords
punct = punctuation+'«»—…“”*№–'
stops = set(stopwords.words('russian'))

def normalize(text):
    
    words = [word.strip(punct) for word in text.lower().split()]
    words = [morph.parse(word)[0].normal_form for word in words if word and word not in stops]

    return words

In [10]:
data['content_norm'] = data['content'].apply(normalize)

In [11]:
data['content_norm_str'] = data['content_norm'].apply(' '.join)

In [41]:
tfidf = TfidfVectorizer(ngram_range=(1,2), min_df=5)

In [42]:
tfidf.fit(data['content_norm_str'])

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=None,
                min_df=5, ngram_range=(1, 2), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

In [43]:
id2word = {i:word for i,word in enumerate(tfidf.get_feature_names())}

In [44]:
texts_vectors = tfidf.transform(data['content_norm_str'])

In [45]:
keywords = [[id2word[w] for w in top] for top in texts_vectors.toarray().argsort()[:,:-11:-1]] 

In [46]:
keywords[:1]

[['обстрел',
  'донбасс',
  'украина',
  'jwplayer',
  'украинский',
  'ga',
  'днр',
  'обсе',
  'x20',
  'location']]

In [47]:
evaluate(data['keywords'], keywords)

Precision -  0.1
Recall -  0.13
F1 -  0.11
Jaccard -  0.06


**Возьмем этот результат за baseline. Precision - 0.1 Recall - 0.13 F1 - 0.11 Jaccard - 0.06**

# **Подход1. Графы**

Попробуем поэкспериментировать с графами

In [35]:
import networkx as nx

In [36]:
from itertools import combinations

In [37]:
def build_matrix(text, window_size=5):
    vocab = set(text)
    word2id = {w:i for i, w in enumerate(vocab)}
    id2word = {i:w for i, w in enumerate(vocab)}
    # преобразуем слова в индексы для удобства
    ids = [word2id[word] for word in text]

    # создадим матрицу совстречаемости
    m = np.zeros((len(vocab), len(vocab)))

    # пройдемся окном по всему тексту
    for i in range(0, len(ids), window_size):
        window = ids[i:i+window_size]
        # добавим единичку всем парам слов в этом окне
        for j, k in combinations(window, 2):
            # чтобы граф был ненаправленный 
            m[j][k] += 1
            m[k][j] += 1
    
    return m, id2word

def some_centrality_measure(text, window_size=5, topn=5):
    
    matrix, id2word = build_matrix(text, window_size)
    G = nx.from_numpy_array(matrix)
    node2measure = dict(nx.degree_centrality(G))   
    return [id2word[index] for index,measure in sorted(node2measure.items(), key=lambda x: -x[1])[:topn]]

Я пробовала разные меры центральности, c degree_centrality выходит лучший результат.

In [38]:
keyword_nx = data['content_norm'].apply(lambda x: some_centrality_measure(x, 10, 10))

In [39]:
evaluate(data['keywords'], keyword_nx)

Precision -  0.12
Recall -  0.15
F1 -  0.13
Jaccard -  0.07


Все метрики немного улучшились. Бейзлайн побит.

# **Подход2. Нормализация**

Попробуем добавить еще стоп-слова и поработать с нормализацией 

In [12]:
!wget https://raw.githubusercontent.com/stopwords-iso/stopwords-ru/master/stopwords-ru.json

--2020-06-19 15:53:20--  https://raw.githubusercontent.com/stopwords-iso/stopwords-ru/master/stopwords-ru.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7349 (7.2K) [text/plain]
Saving to: ‘stopwords-ru.json.2’

stopwords-ru.json.2 100%[===================>]   7.18K  --.-KB/s    in 0s      

2020-06-19 15:53:21 (74.7 MB/s) - ‘stopwords-ru.json.2’ saved [7349/7349]



In [13]:
with open('stopwords-ru.json', 'r') as stw:
    added_stopwords = set(json.load(stw, encoding = 'utf-8')) 

In [14]:
from string import punctuation
from nltk.corpus import stopwords
punct = punctuation+'«»—…“”*№–'
stops = set(stopwords.words('russian'))
stops = stops.union(added_stopwords)

def normalize2(text):
    
    words = [word.strip(punct) for word in text.lower().split()]
    words = [morph.parse(word)[0].normal_form for word in words if word and word not in stops]

    return words

In [15]:
data['content_norm'] = data['content'].apply(normalize2)

In [17]:
data['content_norm'].apply(lambda x: [x[0] for x in Counter(x).most_common(10)]).head(20)

0                           [украина, украинский, сторона, донбасс, обстрел, ситуация, днр, позиция, действие, минский]
1                [противник, учение, условный, сентябрь, российский, войско, военнослужащий, полигон, военный, область]
2                                        [клуб, футболист, команда, игрок, цска, зенит, сезон, один, информация, дзюба]
3                                   [команда, матч, спартак, травма, глушаков, каррер, сезон, клуб, руководство, фанат]
4                                      [оон, rt, чуркин, аббревиатура, известный, мир, сеть, постпред, россия, виталий]
5                            [трамп, расход, миллиард, бюджет, военный, сша, социальный, проект, сокращение, программа]
6                        [военный, операция, сирийский, ракки, исламский, государство, город, сша, пункт, американский]
7        [аляска, америка, русский, компания, российско-американский, резановый, американец, николай, продать, миллион]
8                                       

In [18]:
evaluate(data['keywords'], data['content_norm'].apply(lambda x: [x[0] for x in Counter(x).most_common(10)]))

Precision -  0.12
Recall -  0.16
F1 -  0.13
Jaccard -  0.07


Качество немного улучшилось, бейзлайн побит. Попробуем еще больше нормализовать данные, оставим только существительные и прилагательные.

In [30]:
def normalize3(text):
    words = [word.strip(punct) for word in text.lower().split()]
    words = [morph.parse(word)[0].normal_form for word in words if word and word not in stops]
    words = [morph.parse(word)[0].normal_form for word in words if 'NOUN' in morph.parse(word)[0].tag or 'ADJF' in morph.parse(word)[0].tag]
    return words

In [31]:
data['content_norm3'] = data['content'].apply(normalize3)

In [32]:
data['content_norm3'].apply(lambda x: [x[0] for x in Counter(x).most_common(10)]).head(20)

0                                 [украина, украинский, сторона, донбасс, обстрел, ситуация, позиция, действие, минский, обсе]
1                       [противник, учение, условный, сентябрь, российский, войско, военнослужащий, полигон, военный, область]
2                                               [клуб, футболист, команда, игрок, цска, зенит, сезон, один, информация, дзюба]
3                                          [команда, матч, спартак, травма, глушаков, каррер, сезон, клуб, руководство, фанат]
4                                        [оон, чуркин, аббревиатура, известный, мир, сеть, постпред, россия, виталий, вещание]
5                                       [трамп, расход, миллиард, бюджет, военный, сша, социальный, судак, проект, сокращение]
6                               [военный, операция, сирийский, ракки, исламский, государство, город, сша, пункт, американский]
7         [аляска, америка, русский, компания, российско-американский, резановый, американец, николай, миллион,

In [33]:
evaluate(data['keywords'], data['content_norm3'].apply(lambda x: [x[0] for x in Counter(x).most_common(10)]))

Precision -  0.12
Recall -  0.16
F1 -  0.13
Jaccard -  0.08


Улучшилась метрика Jaccard  по сравнению с предыдущей нормализацией, но совсем немного. Остальные метрики остались прежними. Этот результат тоже больше, чем результаты бейзлайна.

# **Подход3. Готовое решение**

In [25]:
!pip install summa

In [26]:
from summa import keywords
from tqdm import tqdm

In [27]:
summa_keywords = []
for text in tqdm(data['content_norm_str']):
  try:
    kwords = keywords.keywords(text, words=20, language='russian').split('\n')  
  except:
    kwords = ['']
  summa_keywords.append(kwords)

100%|██████████| 4988/4988 [1:21:26<00:00,  1.02it/s]


In [29]:
evaluate(data['keywords'], summa_keywords)

Precision -  0.06
Recall -  0.13
F1 -  0.08
Jaccard -  0.04


Метрики не улучшились, готовое решение справляется плохо. 

# **Подход 4. Gensim**

Попробуем найти фразы с помощью gensim 

In [34]:
from gensim.models.phrases import Phrases, Phraser

In [35]:
def make_phrases(words):
  phrases = Phrases([words], min_count=4, threshold=6)
  bigram = Phraser(phrases)
  cols = bigram[words]
  cols = [word.replace('_', ' ')for word in cols]
  return cols

In [36]:
data['phrases'] = data['content_norm3'].apply(make_phrases)

In [37]:
data['phrases'].head(5)

0            [канцлер, германия, ангел, меркель, ход, брифинг, преддверие, переговоры, украинский, президент, восток, украина, режим, прекращение, огонь, режим, прекращение, огонь, солдат, ситуация, линия, разграничение, беспокойство, меркель, порошенко, украина, минский, договорённость, страна, запад, продление, усиление, антироссийский, санкция, прогресс, реализация, минский, соглашение, донбасс, заявление, меньший, мера, последний, день, беспрецедентный, обострение, обстановка, донбасс, порошенко, председатель, бундестаг, норберт, ламмерота, обстрел, заместитель, начальник, управление, народный, милиция, полковник, эдуард, басурина, брифинг, последний, сутки, январь, украинский, силовик, территория, последний, сутки, артиллерийский, снаряд, калибр, танковый, снаряд, мина, калибр, боеприпас, различный, тип, гранатомёт, стрелковый, оружие, дом, донецк, макеевка, ряд, пункт, обстрел, больница, школа, детский, сад, донецкий, фильтровальный, станция, металлопрокатный, завод, территория, 

In [38]:
key_words = data['phrases'].apply(lambda x: [x[0] for x in Counter(x).most_common(20)])

In [39]:
evaluate(data['keywords'], key_words)

Precision -  0.09
Recall -  0.21
F1 -  0.12
Jaccard -  0.06


Полнота выросла, но остальные метрики - нет.